<a href="https://colab.research.google.com/github/pratikwatwani/Applied-Data-Science/blob/master/Session%2011/Neural%20Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 7_Session 11. Neural networks

The purpose of the below is to classify days over years 2017-2018 by their corresponding mobility patterns between 10 zones in Taipei (quantified by an aggregated temporal network of subway ridership flows across the city)

In [1]:
#use Python 3.7
import numpy as np
import pandas as pd
import keras
from keras.datasets import mnist
from sklearn.preprocessing import MinMaxScaler
from keras.models import Model, Sequential
from keras.layers import Input, Dense
from keras.utils import np_utils

Using TensorFlow backend.


In [0]:
#read the data
TNet=pd.read_csv('https://raw.githubusercontent.com/pratikwatwani/Applied-Data-Science/master/data/taipeiD_TNet2.csv',header=None);

In [3]:
TNet.head() 
#each row represents a 10x10 adjacency matrix of the normalized Taipei subway mobility network between 10 zones flattened into a 100x1 row corresponding to a single day
#days start at jan-1-2017

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,0.017943,0.005415,0.003590,0.008316,0.007859,0.012942,0.012196,0.019543,0.001196,0.003327,0.004588,0.016362,0.003059,0.006420,0.009864,0.005530,0.007357,0.017389,0.000923,0.001672,0.002640,0.002878,0.003133,0.001715,0.003610,0.001157,0.002406,0.004315,0.000550,0.001456,0.008631,0.007123,0.002301,0.010586,0.007468,0.010193,0.010568,0.020925,0.001346,0.002716,...,0.010876,0.007325,0.002859,0.009160,0.013417,0.009071,0.050107,0.043340,0.000679,0.003823,0.013696,0.014299,0.005237,0.015900,0.025870,0.021652,0.035190,0.049923,0.002971,0.009171,0.001081,0.001064,0.000710,0.001091,0.003131,0.008141,0.000839,0.004099,0.009125,0.005163,0.002529,0.001533,0.001860,0.002375,0.005408,0.008922,0.003945,0.011075,0.005073,0.012708
1,0.021283,0.005215,0.003530,0.009359,0.007803,0.014288,0.011185,0.019044,0.001382,0.003499,0.004859,0.016886,0.003053,0.007339,0.009820,0.005745,0.006608,0.016490,0.001023,0.001866,0.002897,0.002929,0.002973,0.001817,0.003471,0.001210,0.002197,0.004039,0.000664,0.001655,0.009672,0.007348,0.002248,0.012551,0.007475,0.011087,0.009090,0.020644,0.001560,0.003032,...,0.010139,0.006609,0.002760,0.008469,0.010956,0.009114,0.046897,0.038464,0.000647,0.003762,0.014380,0.016011,0.003765,0.017911,0.022929,0.021901,0.034270,0.040281,0.003776,0.009128,0.001298,0.001179,0.000663,0.001337,0.003490,0.008978,0.000753,0.004377,0.010360,0.005964,0.002803,0.001757,0.001783,0.002549,0.005515,0.009650,0.003596,0.009618,0.005946,0.013709
2,0.028988,0.006511,0.005591,0.012970,0.007816,0.015878,0.010973,0.015768,0.002252,0.005388,0.006879,0.013790,0.003706,0.009401,0.008878,0.006245,0.006937,0.013613,0.001545,0.002790,0.005575,0.003375,0.004373,0.003305,0.005246,0.001148,0.002829,0.003980,0.000803,0.002643,0.013623,0.008729,0.003722,0.015259,0.006972,0.012822,0.009648,0.017857,0.002672,0.004533,...,0.011613,0.007232,0.003313,0.009770,0.008924,0.009524,0.039863,0.029368,0.000581,0.005011,0.013882,0.013373,0.003620,0.017100,0.018839,0.018666,0.026413,0.030177,0.003673,0.008805,0.002030,0.001531,0.000786,0.002192,0.004388,0.010398,0.000546,0.004129,0.011692,0.009807,0.004649,0.002555,0.002672,0.004291,0.007385,0.009558,0.004293,0.008791,0.010040,0.016301
3,0.029534,0.006471,0.005615,0.013017,0.007717,0.016098,0.011182,0.015815,0.002325,0.005443,0.006955,0.014044,0.003699,0.009330,0.008967,0.006290,0.007313,0.013566,0.001511,0.002833,0.005504,0.003456,0.004210,0.003239,0.005267,0.001098,0.002910,0.003914,0.000742,0.002519,0.013751,0.008552,0.003736,0.014924,0.006757,0.012755,0.009960,0.017484,0.002665,0.004498,...,0.011870,0.007473,0.003513,0.010152,0.009209,0.009930,0.041379,0.029797,0.000618,0.005187,0.013526,0.012225,0.003561,0.016417,0.018527,0.017725,0.025343,0.030699,0.003375,0.007993,0.002014,0.001469,0.000773,0.002228,0.004599,0.010936,0.000596,0.004077,0.012252,0.009988,0.004611,0.002473,0.002636,0.004195,0.007255,0.009487,0.004316,0.008729,0.010296,0.016437
4,0.029333,0.006525,0.005727,0.013098,0.007692,0.016358,0.011000,0.015677,0.002344,0.005527,0.006860,0.013586,0.003710,0.009243,0.008994,0.006580,0.007113,0.014127,0.001536,0.002922,0.005472,0.003366,0.004166,0.003233,0.005255,0.001152,0.002889,0.003851,0.000735,0.002525,0.013695,0.008369,0.003819,0.015103,0.006924,0.012921,0.009824,0.017778,0.002605,0.004648,...,0.011968,0.007428,0.003594,0.010037,0.009058,0.009952,0.040614,0.030371,0.000633,0.005188,0.013355,0.011994,0.003674,0.016204,0.018132,0.017880,0.025106,0.030883,0.003316,0.008219,0.002130,0.001476,0.000823,0.002186,0.004413,0.010712,0.000562,0.004160,0.011789,0.009981,0.004694,0.002515,0.002677,0.004222,0.007269,0.009921,0.004387,0.008923,0.010381,0.016914


In [0]:
#convert to an array and scale the data
X=np.array(TNet);

In [0]:
X=MinMaxScaler(feature_range=(0, 1), copy=True).fit_transform(X)

In [6]:
X.shape

(669, 100)

In [7]:
#define day of the week corresponding to each day of observation; 0-Sunday, 1-Monday,...,6-Saturday
y=np.array(range(669))%7; y[:10]

array([0, 1, 2, 3, 4, 5, 6, 0, 1, 2])

In [8]:
yc=np_utils.to_categorical(y) #get categorical binary variables isSunday, isMonday,...
yc[:5]

array([[1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.]], dtype=float32)

In [0]:
X_test=X[400:,:]; X_train=X[:400,:]; #split the data into training and test
y_test=yc[400:,:]; y_train=yc[:400,:]
dim = X_train.shape[1]

In [10]:
print(X_test.shape)
print(X_train.shape)
print(y_train.shape)
print(y_test.shape)

(269, 100)
(400, 100)
(400, 7)
(269, 7)


## Task 1. Classify weekdays/weekends
Label the rows with ones for weekends, zeros for weekdays.
Train a neural network with 4 layers of 30,10,3 and 1 (output) neurons over the training sample against this label, evaluating its performance over the test sample. Report the acheived accuracy (categorical) over the test sample

First three layers use relu activation function, last one - sigmoid.
Use loss='binary_crossentropy', optimizer='adam', 100 epochs, batch_size=20. 

In [11]:
y

array([0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0,
       1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1,
       2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2,
       3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3,
       4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4,
       5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5,
       6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6,
       0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0,
       1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1,
       2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2,
       3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3,
       4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4,
       5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5,
       6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6,

In [12]:
y_c = y
for idx, item in enumerate(y_c):
  if ((item == 0)| (item == 6)):
    y_c[idx] = 1
  else:
    y_c[idx] = 0

y_c

array([1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,

In [0]:
y1_train=y_c[:400]
y1_test=y_c[400:]

In [14]:
print(y1_train.shape)
print(y1_test.shape)


(400,)
(269,)


In [15]:
np.random.seed(1612)
test = pd.DataFrame()
model = Sequential()
model.add(Dense(30, activation='relu', input_dim=dim))
model.add(Dense(10, activation='relu'))
model.add(Dense(3, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y1_train, validation_data=(X_test, y1_test), epochs=100, batch_size=20, verbose=2)
test = model.predict(X_test)






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 400 samples, validate on 269 samples
Epoch 1/100





 - 0s - loss: 0.4330 - acc: 0.7125 - val_loss: 0.3422 - val_acc: 0.7175
Epoch 2/100
 - 0s - loss: 0.3220 - acc: 0.7125 - val_loss: 0.2880 - val_acc: 0.7175
Epoch 3/100
 - 0s - loss: 0.2805 - acc: 0.7350 - val_loss: 0.2581 - val_acc: 0.7955
Epoch 4/100
 - 0s - loss: 0.2407 - acc: 0.9300 - val_loss: 0.2128 - val_acc: 0.9554
Epoch 5/100
 - 0s - loss: 0.2041 - acc: 0.9525 - val_loss: 0.1830 - val_acc: 0.9554
Epoch 6/100
 - 0s - loss: 0.1761 - acc: 0.9525 - val_loss: 0.1605 - val_acc: 0.9554
Epoch 7/100
 - 0s - loss: 0.1547 - acc: 0.9525 - val_loss: 0.1456 - val_acc: 0.9554
Epoch 8/100
 - 0s - loss: 0.1423 - acc: 0.9525 - val_loss: 0.1371 - val_acc: 0.9554
Epoch 9/100
 - 0s - loss: 0.1370 - acc: 0.9525 - val_loss: 0.1313 - val_acc: 0.9554
Epoch 10/100
 - 0s - loss: 0.1319 - acc: 0.9525 - val_loss: 0.1279 - val_acc: 0.9554
Epoch 

In [16]:
acc_df = pd.DataFrame()
acc_df['predict'] = test.flatten()
acc_df.head()

,predict
0,0.000004
1,0.000009
2,0.000002
3,0.000001
4,0.000044


In [17]:
acc_df['labels'] = y1_test
acc_df.head(10)

,predict,labels
0,0.000004,0
1,0.000009,0
2,0.000002,0
3,0.000001,0
4,0.000044,0
5,0.999995,1
6,0.999994,1
7,0.000007,0
8,0.000015,0
9,0.000657,0


In [18]:
print("Accuracy is: {}\nBUDAM BUM!!".format(1.0*sum((acc_df['labels']==1)==(acc_df['predict']>0.5))/len(acc_df['predict'])))

Accuracy is: 0.9739776951672863
BUDAM BUM!!


## Task 2. Classify all days of the week
Train a neural network against the origial categorical label. Use 5 layers of 40,15,5 and 7 (outputs, representing probabilities for a current input to correspond to each of the weekdays) neurons over the training sample, evaluating its performance over the test sample (use 'categorical_accurary'). Report the acheived accuracy (categorical) over the test sample.

First three layers use relu activation function, last one - sigmoid.
Use loss='binary_crossentropy', optimizer='adam', 200 epochs, batch_size=20

In [19]:
y=np.array(range(669))%7; y[:16]

array([0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1])

In [0]:
y = list(y)
for i, item in  enumerate(y):
  size = y.count(item)
  y[i] = item/size

In [0]:
y2_train=y[:400]
y2_test=y[400:]

In [22]:
np.random.seed(161219)
model = Sequential()
model.add(Dense(40, activation='relu', input_dim=dim))
model.add(Dense(15, activation='relu'))
model.add(Dense(5, activation='relu'))
model.add(Dense(7, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y2_train, validation_data=(X_test, y2_test), epochs=200, batch_size=20, verbose=2)
test2 = model.predict(X_test)

Train on 400 samples, validate on 269 samples
Epoch 1/200
 - 0s - loss: 0.5991 - acc: 0.1450 - val_loss: 0.6297 - val_acc: 0.1413
Epoch 2/200
 - 0s - loss: 0.3747 - acc: 0.1450 - val_loss: 0.7069 - val_acc: 0.1413
Epoch 3/200
 - 0s - loss: 0.2094 - acc: 0.1450 - val_loss: 1.0466 - val_acc: 0.1413
Epoch 4/200
 - 0s - loss: 0.1980 - acc: 0.1450 - val_loss: 1.0311 - val_acc: 0.1413
Epoch 5/200
 - 0s - loss: 0.1957 - acc: 0.1450 - val_loss: 0.9698 - val_acc: 0.1413
Epoch 6/200
 - 0s - loss: 0.1948 - acc: 0.1450 - val_loss: 0.9866 - val_acc: 0.1413
Epoch 7/200
 - 0s - loss: 0.1940 - acc: 0.1450 - val_loss: 0.9789 - val_acc: 0.1413
Epoch 8/200
 - 0s - loss: 0.1932 - acc: 0.1450 - val_loss: 0.9746 - val_acc: 0.1413
Epoch 9/200
 - 0s - loss: 0.1925 - acc: 0.1450 - val_loss: 0.9792 - val_acc: 0.1413
Epoch 10/200
 - 0s - loss: 0.1918 - acc: 0.1450 - val_loss: 0.9767 - val_acc: 0.1413
Epoch 11/200
 - 0s - loss: 0.1913 - acc: 0.1450 - val_loss: 0.9696 - val_acc: 0.1413
Epoch 12/200
 - 0s - loss: 0

In [24]:
acc_df1 = pd.DataFrame()
acc_df1['predict'] = test2.flatten()
acc_df1.head()

,predict
0,0.042751
1,0.043002
2,0.050843
3,0.074965
4,0.104316


In [25]:
acc_df1['labels'] = y2_test
acc_df1.head(10)

,predict,labels
0,0.042751,0.025641
1,0.043002,0.051282
2,0.050843,0.076923
3,0.074965,0.105263
4,0.104316,0.131579
5,0.044384,0.157895
6,0.001490,0.000000
7,0.051737,0.026316
8,0.057636,0.052632
9,0.059519,0.078947


In [27]:
print("Accuracy is: {}\nDone for this semester!!".format(1.0*sum((acc_df1['labels']==1)==(acc_df1['predict']>0.5))/len(acc_df1['predict'])))

Accuracy is: 0.9814126394052045
Done for this semester!!
